# Análises Além Tutorial

<br>

# Carregando Conjunto de Dados

In [1]:
import pandas as pd

dados_balancear = pd.read_csv('dados_balancear.csv', delimiter=',')

# Balanceamento de Classes

In [2]:
#new_data = x
#new_data['mais50k'] = y_true

print(dados_balancear['sex'])

0          Male
1          Male
2          Male
3          Male
4        Female
          ...  
48837    Female
48838      Male
48839    Female
48840      Male
48841    Female
Name: sex, Length: 48842, dtype: object


## Oversampling Sexo

In [3]:
from imblearn.over_sampling import RandomOverSampler

#criando modelo de oversampling
over_sampler = RandomOverSampler(random_state=99)

In [4]:
#realizando reamostragem oversampling, utilizando o sexo como rótulo para o balanceamento
x_over, y_over = over_sampler.fit_resample(dados_balancear.drop(['sex'], axis=1), dados_balancear['sex'])

In [5]:
#concatenando dataframes gerados pelo over sampler, para recolocar o sexo como variável explicativa e mais50k como variável resposta
x_over['sex'] = y_over
dados_over = x_over

dados_over_y = dados_over['mais50k']
dados_over_x = dados_over.drop(['mais50k'], axis=1)

In [6]:
#contagem da realização do oversampling
dados_over_x['sex'].value_counts()

Male      32650
Female    32650
Name: sex, dtype: int64

In [7]:
#convertendo variáveis categóricas
dados_over_x_dummies = pd.get_dummies(dados_over_x)

In [8]:
#função que executa a classificação justa
def executa_classificacao(classificador, x, y, atributo_sensivel):
    paridade_demografica = DemographicParity()

    #criação de um segundo classificador de Árvore de Decisão
    #classificador = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)

    mitigator = ExponentiatedGradient(classificador, paridade_demografica)

    #realizando o treino do modelo justo e utilizando o sexo como atributo sensível
    mitigator.fit(x, y, sensitive_features=atributo_sensivel)

    #previsão dos dados utilizando o modelo justo
    #y_pred_mitigated = mitigator.predict(x)
    return mitigator.predict(x)

#função que calcula as métricas agrupadas
def calcula_metricas(y_true, y_pred, atributo_sensivel):
    #agrupamento de métricas para cálculo com MetricFrame
    metricas = {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'false positive rate': false_positive_rate,
        'true positive rate': true_positive_rate,
        'selection rate': selection_rate,
        'count': count
    }

    return MetricFrame(
        metrics=metricas,
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=atributo_sensivel
    )

In [9]:
from fairlearn.metrics import MetricFrame, selection_rate, false_positive_rate, true_positive_rate, count
from sklearn.metrics import accuracy_score, accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

dados_over_x_dummies, dados_over_y

arvore_decisao = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)

dados_over_prev = executa_classificacao(arvore_decisao, dados_over_x_dummies, dados_over_y, dados_over_x['sex'])

C:\Users\Rodolfo\AppData\Roaming\Python\Python39\site-packages\fairlearn\reductions\_moments\utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
C:\Users\Rodolfo\AppData\Roaming\Python\Python39\site-packages\fairlearn\reductions\_moments\utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
C:\Users\Rodolfo\AppData\Roaming\Python\Python39\site-packages\fairlearn\reductions\_moments\utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series a

In [10]:
#calcula métricas para os dados com oversampling
resultado_oversampling = calcula_metricas(dados_over_y, dados_over_prev, dados_over_x['sex'])

## Undersampling Sexo

In [11]:
from imblearn.under_sampling import RandomUnderSampler

#criando modelo de undersampling
under_sampler = RandomUnderSampler()
exit()
#realizando reamostragem undersampling, utilizando o sexo como rótulo para o balanceamento
x_under, y_under = under_sampler.fit_resample(dados_balancear.drop(['sex'], axis=1), dados_balancear['sex'])



#concatenando dataframes gerados pelo under sampler, para recolocar o sexo como variável explicativa e mais50k como variável resposta
x_under['sex'] = y_under
dados_under = x_under

dados_under_y = dados_under['mais50k']
dados_under_x = dados_under.drop(['mais50k'], axis=1)

#contagem da realização do undersampling
dados_under_x['sex'].value_counts()

#convertendo variáveis categóricas
#dados_under_x_dummies = pd.get_dummies(dados_under_x)

Female    16192
Male      16192
Name: sex, dtype: int64

In [ ]:
#realizando reamostragem undersampling, utilizando o sexo como rótulo para o balanceamento
#x_under, y_under = under_sampler.fit_resample(dados_balancear.drop(['sex'], axis=1), dados_balancear['sex'])

dados_balancear

In [ ]:
exit()
race = dados.data['race']

race.value_counts().sort_values().plot(kind='barh', title='Quantidade de Indivíduos Étnicos')
race.value_counts()

Error: Kernel is dead